# VariTex Demo
This notebook is about [Varitex: Variational Neural Face Textures](https://arxiv.org/pdf/2104.05988.pdf) ([ICCV 2021](https://iccv2021.thecvf.com/)). For more information, please visit the [project page](https://mcbuehler.github.io/VariTex/).

![](https://ait.ethz.ch/people/buehler/public/varitex/teaser.png)

## Preparation
Please follow the instructions in the [README](https://github.com/mcbuehler/VariTex/blob/main/README.md) and download the following files:
- [ ] the model [checkpoint](https://ait.ethz.ch/people/buehler/public/varitex/pretrained.zip)
- [ ] the preprocessed [dataset](https://ait.ethz.ch/people/buehler/public/varitex/dataset_preprocessed.zip)
- [ ] the [Basel Face Model](https://faces.dmi.unibas.ch/bfm/bfm2017.html)
- [ ] the Basel Face Model [UV parameterization](https://github.com/unibas-gravis/parametric-face-image-generator/blob/master/data/regions/face12.json). 



In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
warnings.filterwarnings('ignore')
import pytorch_lightning as pl
import torch
import numpy as np
from varitex.data.keys_enum import DataItemKey as DIK
pl.seed_everything(1234)
from mutil.object_dict import ObjectDict
from torch.nn.functional import normalize as normalizeT



Global seed set to 1234


In [2]:
import sys
sys.path.append('/home/matthias/ETH/Thesis/nFlows/nflows')
#### Nflows
from nflows.flows.base import Flow
from nflows.flows.realnvp import SimpleRealNVP
from nflows.distributions.normal import StandardNormal
from nflows.transforms.coupling import CouplingTransform, AffineCouplingTransform, AdditiveCouplingTransform
from nflows.flows import realnvp
sys.path.append('/home/matthias/ETH/Thesis/nFlows/pytorch-normalizing-flows/')

#### NfLib
# from nflib.flows import (
#     AffineConstantFlow, ActNorm, AffineHalfFlow, 
#     SlowMAF, MAF, IAF, Invertible1x1Conv,
#     NormalizingFlow, NormalizingFlowModel,
# )
# from nflib.spline_flows import NSF_AR, NSF_CL




from torch import distributions
from torch.distributions import MultivariateNormal, Uniform, TransformedDistribution, SigmoidTransform
import itertools

from nflows.flows.base import Flow
from nflows.flows.realnvp import SimpleRealNVP
from nflows.flows.autoregressive import MaskedAutoregressiveFlow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform, MaskedUMNNAutoregressiveTransform, MaskedPiecewiseLinearAutoregressiveTransform,MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation, RandomPermutation
from nflows.transforms.nonlinearities import LeakyReLU
from nflows.transforms.normalization import ActNorm, BatchNorm
from nflows.transforms.coupling import CouplingTransform, AffineCouplingTransform, AdditiveCouplingTransform
from nflows.transforms.standard import PointwiseAffineTransform

import matplotlib.pyplot as plt

In [3]:
Res128_nonorm_path = '/home/matthias/ETH/Thesis/VariTexLocal/VariTex/output/varitex/varitex/Res128_GLO_nonorm/checkpoints/epoch=43-step=377079.ckpt'

ckptFolder ='/home/matthias/ETH/Thesis/VariTexLocal/output/ckpts/Res128_GLO_NoNorm'
filename = 'epoch=43-step=377079.ckpt'
Res128_nonorm_path = os.path.join(ckptFolder, filename)
Res128_norm1_path = '/home/matthias/ETH/Thesis/VariTexLocal/output/forDemo/1vc1xbec/checkpoints/epoch=43-step=377079.ckpt'
nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows'
nfCKPT = 'young-sun-6.ckpt'
nfPath = os.path.join(nfFolderPath,nfCKPT)
opt = {

        #"checkpoint": os.path.join(os.getenv("CP", "pretrained/ep44.ckpt")),
        "checkpoint": Res128_nonorm_path,
        "dataroot_npy": os.path.join(os.getenv("DP", ""), 'FFHQ/preprocessed_dataset_new'),
        "path_bfm": os.path.join(os.getenv("FP", ""), "basel_facemodel/model2017-1_face12_nomouth.h5"),
        "path_uv": os.path.join(os.getenv("FP", ""), "basel_facemodel/face12.json"),
        "device": "cuda"
        }

path_latent = os.path.join(os.getenv("DP"), "FFHQ/preprocessed_dataset_new/latents.npy")

from varitex.demo import Demo
opt.update(use_NF=False)
demo = Demo(opt)

Loading BFM 2017 into GPU... (this can take a while)
Done


### Loading Latent Distributions
The variables `latent_mean` and `latent_std` contain the mean and standard deviations of all samples from the holdout set, predicted by the encoder. You can choose the index of the distribution that you want to render, or you can sample from these distributions.


Example indices from the holdout set used the [paper](https://arxiv.org/pdf/2104.05988.pdf):
* [Teaser image](https://ait.ethz.ch/people/buehler/public/varitex/teaser.png) (Fig. 1): 184 (female), 33 (male).
* Fig. 3: 8 (Identity 1), 904 (identity 2).
* Fig. 5: Sampled from distributions 184, 148, 313, 319.

latent_distributions = np.load(path_latent)
latent_mean = torch.Tensor(latent_distributions[:,0])
latent_std = torch.Tensor(latent_distributions[:,1])
print("{} distributions loaded.".format(latent_mean.shape[0]))

# VariTex Controls

![](https://ait.ethz.ch/people/buehler/public/varitex/varitex_pipeline_inference.png)

The generator $G$ provides
    a) __neural control__ over identity and appearance via a latent code `z` (consisting of two parts $z_{face}$ and $z_{additive}$) and 
    b) __graphical control__ over geometry (shape `sp`, expression `ep`, and pose `R`) via the parameters of a 3D morphable face model. For simplicity, let's define the pose as euler angles `theta` ($\theta$).
    
$$G (z, sp, ep, R(\theta))$$

* `z` $\in \mathbb{R}^{256}$: Latent identity code.
* `sp` $\in \mathbb{R}^{199}$ and `ep` $\in \mathbb{R}^{100}$: Shape and expression coefficients of the Basel Face Model. They define the geometry of the rendered face. Zeros yield a neutral face.
* `theta` $\in \mathbb{R}^3$: Pose in euler angles (degrees). Zeros yield a frontal face.

## Graphical Control: Pose and Expression

### Shape and Expression
We load shape `sp` and expression `ep` coefficients from the FFHQ validation set. Choose any other index to load different shapes and expressions from the validation set. These parameters could also be sampled from the Basel Face Model.

Change the indices for `sp` and `ep` to modify the facial expression, and shape.

In [ ]:
import copy

In [ ]:
pcapath = '/home/matthias/ETH/Thesis/VariTexLocal/datasets/pcaLatents.npy'
pca = np.load(pcapath).astype('float')
pcaN = (pca[:,:128])
print(np.linalg.norm(pcaN,axis=1))
usedpca = pcaN/np.linalg.norm(pcaN,axis=1).reshape((-1,1))
print(np.linalg.norm(usedpca,axis=1))
print(usedpca.shape)
mean = usedpca.mean(1).mean(0)
std = usedpca.std(1).mean(0)
normalDist = np.random.normal(mean, std, (70000,128))
print(usedpca.std(1).mean(0)*100000.)
print(normalDist.std(1).mean(0)*100000.)
plt.hist([normalDist.flatten(),usedpca.flatten()],bins = 100,label=['G','x'], histtype='stepfilled')
plt.legend()
plt.show()

In [ ]:


plt.hist(stdNormal,bins = 100,color='r')
plt.show()

In [ ]:
standard = torch.randn(100000).numpy().flatten()
mu=1.
sigma=1.0
normal = torch.normal(mu,sigma,size = [100000]).numpy().flatten()
plt.hist([standard, normal],bins = 100,color=['r','b'], histtype='stepfilled')
plt.grid()
plt.show()

# Random sampling

## Linear

In [ ]:
shapes, expressions = demo.load_shape_expressions()
index_id, index_sp, index_ep = 184, 184, 184
sp = shapes[index_sp].unsqueeze(0)
ep = expressions[index_ep].unsqueeze(0)
theta = torch.Tensor((0,-35,0))
z_sampled = (normalizeT(torch.randn(10, 256),dim=1)*1)


norm = 0
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
demo.to_image(batches_out)

## Spherical

In [ ]:
norm = 1
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
print(torch.linalg.norm(z_sampled, dim=1))
demo.to_image(batches_out)

In [ ]:
norm = 2
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
print(torch.linalg.norm(z_sampled*norm, dim=1))
demo.to_image(batches_out)

In [ ]:
norm = 3
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
demo.to_image(batches_out)

In [ ]:
norm = 4
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
demo.to_image(batches_out)

In [ ]:
norm = 5
batches_out = [demo.runGlo(z=z.unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_sampled]
demo.to_image(batches_out)

In [ ]:
norm = 6
batches_out = [demo.runGlo(z=z.unsqueeze(0)*6, sp=sp, ep=ep, theta=theta) for z in z_sampled]
demo.to_image(batches_out)

# Random interpolation

In [ ]:
from mutil.np_util import interpolation
n_samples = 10

norm = 3
zA, zB = (normalizeT(torch.randn(256),dim=0)*norm), (normalizeT(torch.randn(256),dim=0)*norm) 
z_interpolated = torch.Tensor(interpolation(n_samples, zA.numpy(), zB.numpy()))
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in z_interpolated]
print("Norms: ", torch.linalg.norm(z_interpolated, dim=1))
demo.to_image(batches_out)

In [ ]:
batches_out = [demo.runGlo(z=normalizeT(z, dim=0).unsqueeze(0)*norm, sp=sp, ep=ep, theta=theta) for z in z_interpolated]
demo.to_image(batches_out)

# Zero to norm n interpolation

In [ ]:
n_samples = 10

norm = 11
zA, zB = (torch.zeros(256).unsqueeze(0)), (normalizeT(torch.randn(256),dim=0).unsqueeze(0)*norm) 
z_interpolated = torch.Tensor(interpolation(n_samples, zA.numpy(), zB.numpy()))
batches_out = [demo.runGlo(z=z, sp=sp, ep=ep, theta=theta) for z in z_interpolated]
demo.to_image(batches_out)

In [ ]:
latent = demo.pipeline.Z.weight
latent = latent[:59990]
latent_training_codes = latent[21:31]
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in latent_training_codes]
print("Norms: ", torch.linalg.norm(latent_training_codes,dim=1))
demo.to_image(batches_out)

In [ ]:
latent_training_codes = normalizeT(latent_training_codes, dim=1)*2.5
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in latent_training_codes]
print("Norms: ", torch.linalg.norm(latent_training_codes,dim=1))
demo.to_image(batches_out)

In [ ]:
import matplotlib.pyplot as plt
def show_hist_full(i):
    HIST_BINS = np.linspace(-1.5, 1.5, 100)
    plt.hist(latent.detach().cpu().numpy().flatten(), HIST_BINS,alpha=1)
    filename = os.path.join('output/hists/full','hist_full_epoch')
    #plt.savefig(filename+str(i)+'full.jpg')
    plt.show()
    
def show_hist(i):
    HIST_BINS = np.linspace(-1.5, 1.5, 100)
    plt.hist(latent[i,:128].detach().cpu().numpy().flatten(), HIST_BINS,alpha=1)
    plt.hist(latent[i,128:].detach().cpu().numpy(), HIST_BINS,alpha=0.6)
    plt.title('Latent code: '+str(i))
    filename = os.path.join('output/hists','hist_epoch')
    plt.savefig(filename+str(i)+'.jpg')
    plt.show()

In [ ]:
print(torch.mean(latent,dim=1).mean().item())

In [ ]:
latent = torch.load(Res128_nonorm_path)['state_dict']['Z.weight']

# Normalizing Flows (extra)

# Use NflowLib

In [ ]:
latent = demo.pipeline.Z.weight
latent = latent[:59990]

In [ ]:
def getTransform(transform, config):
    ##Permutations
    if(transform == 'randomPermutation'):
        return RandomPermutation(features=config['num_features'])
    elif(transform == 'reversePermutation'):
        return ReversePermutation(features=config['num_features'])
    ##Autoregressions
    elif(transform == 'maskedAffineAR'):
        return MaskedAffineAutoregressiveTransform(features=config['num_features'],
                                                   hidden_features=config['hidden_features'])
    elif(transform == 'maskedAffineARUMNN'):
        return MaskedUMNNAutoregressiveTransform(features=config['num_features'],
                                                 hidden_features=config['hidden_features'])
    elif(transform == 'maskedAffineARLinear'):
        return MaskedPiecewiseLinearAutoregressiveTransform(features=config['num_features'],
                                                         hidden_features=config['hidden_features'],
                                                        num_bins = 10)
    elif(transform == 'maskedAffineARQuadratic'):
        return MaskedPiecewiseQuadraticAutoregressiveTransform(features=config['num_features'],
                                                        hidden_features=config['hidden_features'],
                                                        num_bins = 10)
    elif(transform == 'actNorm'):
        return ActNorm(features=config['num_features'])
    elif(transform == 'batchNorm'):
        return BatchNorm(features=config['num_features'])
    else:
        print("Transform "+ transform +" is not supported")


def getFlow(config):
    modelName = config['model']
    base_dist = StandardNormal(shape=[config['num_features']])
    if(not isinstance(modelName,list)):
        if(modelName=='simpleRNVP'):
            flow = realnvp.SimpleRealNVP(
                    features = config['num_features'], 
                    hidden_features = config['hidden_features'], 
                    num_layers = config['num_layers'], 
                    num_blocks_per_layer = config['num_blocks_per_layer'],
                    activation = config['activation'],
                    use_volume_preserving = config['use_volume_preserving'],
                    batch_norm_between_layers = config['batch_norm_between_layers'], 
                    batch_norm_within_layers = config['batch_norm_within_layers'],
                    dropout_probability = config['dropout_probability'] )
            return flow
        elif(modelName == 'made'):
            flow = MaskedAutoregressiveFlow(
                    features = config['num_features'], 
                    hidden_features = config['hidden_features'], 
                    num_layers = config['num_layers'], 
                    num_blocks_per_layer = config['num_blocks_per_layer'])
            return flow
    else:
        transforms = []
        for _ in range(config['num_layers']):
            for layer in modelName:
                transforms.append(getTransform(layer,config))     
    transform = CompositeTransform(transforms)
    flow = Flow(transform, base_dist)
    return flow

def getAllFlows(expPath, exp_name, maxEpoch):
    models = []
    for i in range(maxEpoch):
        currentPath = os.path.join(expPath, exp_name+str(i)+'.ckpt')
        currentDicts = torch.load(currentPath)
        config = currentDicts['config']
        model = getFlow(config)
        model.load_state_dict(currentDicts['model_state_dict'])
        models.append(model)
    return models
        
def getBatches(model, nImages=10):
    model.eval()
    shapes, expressions = demo.load_shape_expressions()
    index_id, index_sp, index_ep = 184, 184, 184
    index_id, index_sp, index_ep = 184, 184, 184
    sp = shapes[index_sp].unsqueeze(0)
    ep = expressions[index_ep].unsqueeze(0)
    theta = torch.Tensor((0,-35,0))
    samples = model.sample(nImages)
    samples = samples.detach()
    batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in samples]
    printGParams(samples.numpy(), "current Model")
    return batches_out

In [ ]:
nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows/output'
exp_name = 'blooming-universe-33'
expPath = os.path.join(nfFolderPath,exp_name)
models = getAllFlows(expPath, exp_name, 37)
i=0
batches_out = getBatches(models[i],10)
demo.to_image(batches_out)

In [ ]:
i=10
batches_out = getBatches(models[i],10)
demo.to_image(batches_out)

In [ ]:
i=20
batches_out = getBatches(models[i],10)
demo.to_image(batches_out)

In [ ]:
i=30
batches_out = getBatches(models[i],10)
demo.to_image(batches_out)

In [ ]:
for i in models:
    makeImages(i)

In [ ]:
nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows/output'
exp_name = 'golden-energy-32'
expPath = os.path.join(nfFolderPath,exp_name)
cepoch = 5
nfPath = os.path.join(expPath, exp_name+str(cepoch)+'.ckpt')

In [ ]:
flowDicts = torch.load(nfPath)
print(flowDicts.keys())
config = flowDicts['config']
print(config)
flow = getFlow(config)
model = flow
model.load_state_dict(flowDicts['model_state_dict'])
flowDicts['epoch']

In [ ]:
def printGParams(l, name):
    print('Mean '+name+':', "{:.8E}".format(l.mean(1).mean().astype('float')))
    print('Std '+name+':', "{:.8E}".format(l.std(1).mean().astype('float')))
    print('Norm '+name+':', "{:.8E}".format(np.linalg.norm(l, axis=1).mean().astype('float')))


In [ ]:
printGParams(latent.detach().cpu().numpy(),'x')

In [ ]:
model.eval()

fold = model.sample(10)
fold = fold.detach().numpy()

exLatent = latent[:10,:256].cpu()
pLatent = model._transform(exLatent)
exLatent = exLatent.detach().numpy()
pLatent = pLatent[0].detach().numpy()

plt.hist(exLatent.flatten(),color='b',bins=100, alpha=1,label='x')
plt.hist(pLatent.flatten(),color='r',bins=100, alpha=0.7,label='x->z')
#plt.hist(fnew,bins=100,alpha=0.5)
plt.legend()
plt.show()
printGParams(exLatent,'x')
printGParams(pLatent,'z')

In [ ]:
n = 100
# fold = model.sample(n)
# fold = fold.detach().numpy()
exLatent = latent[:n,:256].cpu()
exLatent = exLatent.detach().numpy()
#plt.hist(exLatent.flatten(),color='b',bins=100, alpha=1,label='x')
#plt.hist(fold.flatten(),color='r',bins=100, alpha=0.5,label='z->x')
#plt.hist(pLatent.flatten(),color='r',bins=100, alpha=0.7,label='x->z')
fold = np.random.normal(exLatent.mean(1).mean(0), exLatent.std(1).mean(0), size=(100,256))
plt.hist([fold.flatten(),exLatent.flatten()], bins=100, alpha=0.7,label=['G','x'], stacked=False, histtype='stepfilled')
#plt.hist(fnew,bins=100,alpha=0.5)
plt.legend()
plt.show()
printGParams(exLatent,'x')
printGParams(fold,'x sampled')

In [ ]:
fold = model.sample(10)
fold = fold.detach().numpy()

plt.hist(fold.flatten(),color='b',bins=100, alpha=1,label='z->x')
#plt.hist(pLatent.flatten(),color='r',bins=100, alpha=0.7,label='x->z')
#plt.hist(fnew,bins=100,alpha=0.5)
plt.legend()
plt.show()

In [ ]:
fold = model._distribution.sample(10)
fold = fold.detach().numpy()

plt.hist(fold.flatten(),color='b',bins=100, alpha=1,label='z')
#plt.hist(pLatent.flatten(),color='r',bins=100, alpha=0.7,label='x->z')
#plt.hist(fnew,bins=100,alpha=0.5)
plt.legend()
plt.show()
printGParams(fold,'prior')

In [ ]:
zs = model.sample(100)
z = zs.detach().numpy()
print(z.shape)
z = z.flatten()

shapes, expressions = demo.load_shape_expressions()
index_id, index_sp, index_ep = 184, 184, 184
sp = shapes[index_sp].unsqueeze(0)
ep = expressions[index_ep].unsqueeze(0)
theta = torch.Tensor((0,-35,0))


z_sampled = (normalizeT(torch.randn(10, 256),dim=1)*1)
samples = zs.detach()
# normsSampled.append(torch.norm(samples,dim=1).mean().item())
# normsSampled.append(torch.norm(samples,dim=1).mean().item())
print('Groundtruth',torch.norm(latent[:1000].detach(),dim=1).mean())
print('NF Latent sampled', torch.norm(samples,dim=1).mean())
norm = 0
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in samples[:10]]
demo.to_image(batches_out)

In [ ]:
for i in exLatent[:5]:
    plt.plot(i)
plt.grid()
plt.show()

# Use NFLib

In [ ]:
normsSampled = []

In [ ]:
nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows/output'
exp_name = 'feasible-fire-18'
nfPath = os.path.join(nfFolderPath,exp_name)
flowArray = [] 
i=0
while(True):
    filename = os.path.join(nfPath, exp_name+str(i)+'.ckpt')
    i+=1
    if(os.path.isfile(filename)):
        file = np.load(filename)
        flowArray.append(file)
    else:
        break

In [ ]:
def loadDicts(dicts):
    modelArray = []
    for i in dicts:
        model = 

In [ ]:
nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows'
nfCKPT = 'spring-plant-8.ckpt'
nfCKPT = 'divine-universe-10.ckpt' 
filename = 'fallen-cosmos-12'
filename = 'dainty-thunder-13'
filename = 'denim-voice-15'

nfFolderPath = '/home/matthias/ETH/Thesis/nFlows/nflows/output'
exp_name = 'feasible-fire-18'
exp_name = 'flowing-fire-19'
exp_name = 'silver-totem-20'
exp_name = 'crimson-bird-21'
exp_name = 'solar-yogurt-28'
nfPath = os.path.join(nfFolderPath,exp_name)
cepoch = 1
nfPath = os.path.join(nfPath, exp_name+str(cepoch)+'.ckpt')

## load Nf
flowDicts = torch.load(nfPath)
print(flowDicts.keys())
config = flowDicts['config']
print(config.keys())
nlayers = config['num_layers']
hiddenFeatures = config['hidden_features']
latentDim = 256
prior = MultivariateNormal(torch.zeros(latentDim), torch.eye(latentDim))

#prior = TransformedDistribution(Uniform(torch.zeros(latentDim), torch.ones(latentDim)), SigmoidTransform().inv) # Logistic distribution
flows = [AffineHalfFlow(dim=latentDim, parity=i%2, nh=hiddenFeatures) for i in range(nlayers)]
#norms = [ActNorm(dim=latentDim) for _ in flows]
#flows = list(itertools.chain(*zip(norms, flows)))

# flows = [Invertible1x1Conv(dim=latentDim) for i in range(nlayers)]
# norms = [ActNorm(dim=latentDim) for _ in flows]
# couplings = [AffineHalfFlow(dim=latentDim, parity=i%2, nh=hiddenFeatures) for i in range(len(flows))]
# flows = list(itertools.chain(*zip(norms, flows, couplings))) # append a coupling layer after each 1x1
    
    
model = NormalizingFlowModel(prior, flows)
model.load_state_dict(flowDicts['model_state_dict'])
print('Epoch: ',flowDicts['epoch'])

In [ ]:
model.eval()

p = model.prior.sample([10])#.squeeze()
print(p.shape)
new, logdet = model.backward(p)
fnew = new[-1].detach().numpy()
plt.hist(p.detach().numpy().flatten(),color='r',bins=100, alpha=1,label='nflib prior')
plt.hist(fnew.flatten(),bins=100,alpha=0.5, color='b', label = 'z->x')
plt.legend()
plt.show()

printGParams(p.detach().numpy(),'nflib prior')
printGParams(fnew,'nflib z->x')

In [ ]:
zs = model.sample(10)
z = zs[-1]
zf = z.detach().numpy()
z = zf.flatten()
#plt.hist(z, color='r',alpha=1,bins=100, label='z->x')
plt.hist([z,exLatent.flatten()], bins=100, alpha=0.7,label=['z->x','x'], stacked=False, histtype='stepfilled')
#plt.hist([(exLatent+noise.numpy()).flatten(),exLatent.flatten()], bins=100, alpha=0.7,label=['z->x','x'], stacked=False, histtype='stepfilled')
plt.legend()
plt.show()

printGParams(zf,'Sampled')
printGParams(exLatent,'Groundtruth')

In [ ]:
import matplotlib.pyplot as plt
l = latent[:1000].detach().cpu().numpy().flatten()
flow.eval()
f = flow._transform(flow.sample(1000))[0].detach().numpy().flatten()
plt.hist(l,color='r',bins=100)
plt.hist(f,bins=100,alpha=0.5)
plt.show()
print('Mean latent:', l.mean().astype('float'))
print('Mean flow sample:',f.mean().astype('float'))
print('Std latent:',l.std().astype('float'))
print('Std flow sample:',f.std().astype('float'))

In [ ]:
zs = model.sample(100)
z = zs[-1]
z = z.detach().numpy()
print(z.shape)
z = z.flatten()

shapes, expressions = demo.load_shape_expressions()
index_id, index_sp, index_ep = 184, 184, 184
sp = shapes[index_sp].unsqueeze(0)
ep = expressions[index_ep].unsqueeze(0)
theta = torch.Tensor((0,-35,0))
samples1 = flow.sample(10)
samples2 = flow.sample(10)
z_sampled = (normalizeT(torch.randn(10, 256),dim=1)*1)
samples =samples1/6
#samples = normalizeT(samples,dim=1)*3

z_sampled = (normalizeT(torch.randn(10, 256),dim=1)*1)
print(torch.norm(z_sampled,dim=1).mean())
print('Generator Latent: ',torch.norm(samples,dim=1).mean().detach())
samples = zs[-1].detach()
normsSampled.append(torch.norm(samples,dim=1).mean().item())
print('NF Latent sampled', torch.norm(samples,dim=1).mean())
norm = 0
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in samples[:10]]
demo.to_image(batches_out)

In [ ]:
plt.plot(normsSampled)

In [ ]:
print(normsSampled)

# Non GLO demo

In [ ]:
index_id, index_sp, index_ep = 184, 184, 184

shapes, expressions = demo.load_shape_expressions()
sp = shapes[index_sp].unsqueeze(0)
ep = expressions[index_ep].unsqueeze(0)

z = latent_mean[index_id].unsqueeze(0)
theta = torch.Tensor((0,-35,0))

batch_out = demo.run(z=z, sp=sp, ep=ep, theta=theta)
demo.to_image(batch_out)

Running multiple expressions...

In [ ]:
ep_indices = list(range(5))
batches_out = [demo.run(z=z, sp=sp, ep=expressions[ep_index].unsqueeze(0), theta=theta) for ep_index in ep_indices]
demo.to_image(batches_out)

### Pose
Pose can be defined as euler axes (degrees) `theta=(pitch, yaw, roll)`. 

* pitch > 0 $\rightarrow$ looking up
* yaw > 0 $\rightarrow$ looking left
* roll > 0 $\rightarrow$ rotate clockwise

In [ ]:
theta = torch.Tensor((-15, -20, 0))

batch_out = demo.run(z=z, sp=sp, ep=ep, theta=theta)
demo.to_image(batch_out)

In [ ]:
theta_all = [torch.Tensor((0, -t, 0)) for t in 
             np.arange(0, 46, 15)]

batches_out = [demo.run(z=z, sp=sp, ep=ep, theta=t) for t in theta_all]
demo.to_image(batches_out)

## Neural Control: Identity

### Sampling
VariTex controls identity via a latent code `z`. You can generate variants of this identity by resampling the latent code. Execute the cell below multiple times to see different outcomes.

You can change the variable `multiplier` to sample in more/less extreme regions of the learned distribution.

In [ ]:
index = 184
multiplier = 1.5
n_samples = 5
z_list = torch.distributions.Normal(latent_mean[index].clone(), latent_std[index].clone() * multiplier).rsample((n_samples,)) 
theta = torch.Tensor((0, 0, 0))

batches_out = [demo.run(z=z_l.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z_l in z_list]
demo.to_image(batches_out)

### Interpolation
Interpolation between two latent codes smoothly transitions from one identity to another.

In [ ]:
from mutil.np_util import interpolation
n_samples = 5
indexA, indexB = 313, 184

zA, zB = latent_mean[indexA], latent_mean[indexB]
z_interpolated = torch.Tensor(interpolation(n_samples, zA.numpy(), zB.numpy()))

batches_out = [demo.run(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in z_interpolated]
demo.to_image(batches_out)

In [ ]:
z = latent[:3]
print(z.shape)
z0, z1 = z[:,::2], z[:,1::2]
print(z0.shape)
print(z1.shape)

In [ ]:
pathNorms = os.getenv("OP")
pathNorms = os.path.join(pathNorms, 'norm_snapshots')
latentsNF = []
for i in range(15):
    filename = 'norms_epoch_'+str(i)
    file = os.path.join(pathNorms,filename)
    latentNF = np.load(file+'.npy')
    latentsNF.append(latentNF)

In [ ]:
for i in range(15):
    plt.hist(latentsNF[i][:59990].flatten(),bins=100, color='b')
    saveName = os.path.join(pathNorms,'hist'+str(i)+'.png')
    plt.ylim(0,5000)
    plt.xlim(0.5,3.0)
    plt.savefig(saveName)
    plt.show()

In [ ]:
from varitex.ppl_metric import dnnlib_utils

sampling = 'end'
p = torch.rand(20) * (1 if sampling == 'full' else 0)
print(p.unsqueeze(1).unsqueeze(1).shape)

device = torch.device('cuda:0')

print(device)

vgg16_url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metrics/vgg16.pt'
with dnnlib_utils.open_url(vgg16_url, verbose=1) as f:
    f_detector = torch.jit.load(f).eval().to(device)

imgEx = batches_out[0][DIK.IMAGE_OUT].to(device)
out = f_detector(imgEx,resize_images=False, return_lpips=True)
print(out.shape)

f_detector.eval()
f_detector.

def slerp(a, b, t):
    a = a / a.norm(dim=-1, keepdim=True)
    b = b / b.norm(dim=-1, keepdim=True)
    d = (a * b).sum(dim=-1, keepdim=True)
    p = t * torch.acos(d)
    c = b - d * a
    c = c / c.norm(dim=-1, keepdim=True)
    d = a * torch.cos(p) + c * torch.sin(p)
    d = d / d.norm(dim=-1, keepdim=True)
    return d

class PPLSampler(torch.nn.Module):
    def __init__(self, model='encoder', epsilon= 1e-4, interpolation = 'linear', sampling, crop, vgg16):
    def samplePPL():
    #StyleGAN takes generator, epsilon, space, sampling = end/full crop, vgg
    # 1. Randomly choose latents and interpolations
    if(model=='encoder'):
        
    elif(model=='GLO'):
        
    else:
        #model with NF
    # 2. Interpolate
    if(interpolation=='linear'):
        
    else: 
    # 3. Generate images
    
    # 4 Calc Lpips and distance
    lpips_t0, lpips_t1 = self.vgg16(img, resize_images=False, return_lpips=True).chunk(2)
    dist = (lpips_t0 - lpips_t1).square().sum(1) / self.epsilon ** 2
    
    
    
def computePPL()


_feature_detector_cache = dict()
def get_feature_detector(url, device=torch.device('cpu'), num_gpus=1, rank=0, verbose=False):
    assert 0 <= rank < num_gpus
    key = (url, device)
    if key not in _feature_detector_cache:
        is_leader = (rank == 0)
        if not is_leader and num_gpus > 1:
            torch.distributed.barrier() # leader goes first
        with dnnlib.util.open_url(url, verbose=(verbose and is_leader)) as f:
            _feature_detector_cache[key] = torch.jit.load(f).eval().to(device)
        if is_leader and num_gpus > 1:
            torch.distributed.barrier() # others follow
    return _feature_detector_cache[key]

print(torch.randn([3 * 2, 256], device=device).chunk(2)[0].shape)


# Calculate FIDs

## Standard (Training)

## Interpolated

In [ ]:
from torch.utils.data import DataLoader
opt.update({"dataset": 'FFHQ',
            "dataroot_npy": os.path.join(os.getenv("DP"), 'FFHQ/preprocessed_dataset_new'),
            "image_folder": os.path.join(os.getenv("DP"), 'FFHQ/images'),
            "transform_mode": 'all',
            "image_h": 128,
            "image_w": 128,
            "batch_size": 7,
            "num_workers": 12,
            "semantic_regions": list(range(1, 16)),
            "keep_background": False,
            "bg_color" : 'black',
            "latent_dim": 256
                        })
opt_new = ObjectDict(opt)

from varitex.data.npy_dataset import NPYDataset
train_dataset = NPYDataset(opt_new, split="train", augmentation=False)
train_dataloader = DataLoader(train_dataset, batch_size=opt_new.batch_size, num_workers=opt_new.num_workers,
                                      shuffle=False, drop_last=True)

ckptFolder ='/home/matthias/ETH/Thesis/VariTexLocal/output/ckpts/Res128_GLO_NoNorm'
filename = 'epoch=43-step=377079.ckpt'
Res128_nonorm_path = os.path.join(ckptFolder, filename)
Z = np.load(Res128_nonorm_path)
for batch in train_dataloader:
    print(batch.keys())
    print(batch[DIK.IMAGE_IN].shape)
    imgT = batch[DIK.IMAGE_IN]
    #imgT = [k for k in imgT]
    demo.visualizer_complete.tensor2image(imgT[0].squeeze(), return_format='pil')
    break
demo.visualizer_complete.tensor2image(imgT[0].squeeze(), return_format='pil')

In [ ]:
sp = batch[DIK.COEFF_SHAPE][0].unsqueeze(0)
ep = batch[DIK.COEFF_EXPRESSION][0].unsqueeze(0)
# shapes, expressions = demo.load_shape_expressions()
# index_id, index_sp, index_ep = 184, 184, 184
# sp = shapes[index_sp].unsqueeze(0)
# ep = expressions[index_ep].unsqueeze(0)
theta = torch.Tensor((0,-100,0))
Z = demo.pipeline.Z.weight
Z = Z[:59990]
batches_out = [demo.runGlo(z=z.unsqueeze(0), sp=sp, ep=ep, theta=theta) for z in Z[:10]]
demo.to_image(batches_out)

In [ ]:
class LitModel(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)

for batch in train_dataloader:
    print(batch)

data = train_dataset.data["images"][0]
data  = np.array(data).astype('int')
plt.imshow(data)

In [4]:
model_names=['default', 'norm', 'nonorm', 'nf_glo_alternate', 'nf_glo_joint']
def getModel(modelName, opt):
    mp_path = '/cluster/project/infk/hilliges/koenigma/Final_Models'
    mp_path = '/home/matthias/ETH/Thesis/Final_Models'
    if (modelName == 'default'):
        folder = os.path.join(mp_path, 'default/checkpoints')
        opt.update({"checkpoint": os.path.join(folder, 'epoch=43-step=377079.ckpt'),
                    "use_glo": False,
                    "use_NF": False,
                    "alternate": False,
                    "experiment_name": "eval_Default"})
    elif (modelName == 'norm'):
        folder = os.path.join(mp_path, 'GLO_Norm/checkpoints')
        opt.update({"checkpoint": os.path.join(folder, 'epoch=43-step=377079.ckpt'),
                    "use_glo": True,
                    "use_NF": False,
                    "alternate": False,
                    "experiment_name": "eval_norm"})
    elif (modelName == 'nonorm'):
        folder = os.path.join(mp_path, 'GLO_NoNorm/checkpoints')
        opt.update({"checkpoint": os.path.join(folder, 'epoch=43-step=377079.ckpt'),
                    "use_glo": True,
                    "use_NF": False,
                    "alternate": False,
                    "experiment_name": "eval_nonorm"})
    elif (modelName == 'nf_glo_alternate'):
        folder = os.path.join(mp_path, 'GLO_NF_ALTERNATE')
        opt.update({"checkpoint": os.path.join(folder, 'epoch=35-step=308519.ckpt'),
                    "use_glo": True,
                    "use_NF": True,
                    "alternate": True,
                    "experiment_name": "eval_nf_glo_alternate"})
    elif (modelName == 'nf_glo_joint'):
        folder = os.path.join(mp_path, 'GLO_NF_joint/Res128_GLO_NF_lambdae-3')
        opt.update({"checkpoint": os.path.join(folder, 'epoch=43-step=377079.ckpt'),
                    "use_glo": True,
                    "use_NF": True,
                    "alternate": False,
                    "experiment_name": "eval_nf_glo_joint"})
    else:
        print('Model not recognized!')
    return opt

In [5]:
from varitex.evaluation.inference import inference_ffhq
from varitex import validation
#def do_validation(model_name, opt):
name = 'default'
opt.update({"dataset": 'FFHQ',
        "dataroot_npy": os.path.join(os.getenv("DP"), 'FFHQ/preprocessed_dataset_new'),
        "image_folder": os.path.join(os.getenv("DP"), 'FFHQ/images'),
        "transform_mode": 'all',
        "image_h": 128,
        "image_w": 128,
        "batch_size": 1,
        "num_workers": 12,
        "semantic_regions": list(range(1, 16)),
        "keep_background": False,
        "bg_color" : 'black',
        "latent_dim": 256,
        "texture_dim" : 128,
        "texture_nc": 16,
        "nc_feature2image": 64,
        "feature2image_num_layers": 5,
        "use_glo": False,
        "glo_init":'pca',
        "pca_file": os.path.join(os.getenv("BP"), 'datasets/pcaLatents.npy'),
        "experiment_name": 'eval_NoNorm',
        "use_NF": False,
        "eval": True})
opt = getModel(name,opt)
opt_new = ObjectDict(opt)
vals = validation.Validation(opt)

Datasets Loaded
Loading BFM 2017 into GPU... (this can take a while)
Done


In [31]:
import json
def printJSON(jsonPath):
    #jsonPath = os.path.join(jsonPath, 'standards.json')
    try:
        f = open(jsonPath,)        
    except IOError:
        return 'NaN'
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    print(data)
    return data['FID']

In [32]:
paths = []
finalPath = '/home/matthias/ETH/Thesis/Euler/Final_Models'
paths.append(os.path.join(finalPath,'default/checkpoints'))
paths.append(os.path.join(finalPath,'GLO_Norm/checkpoints'))
paths.append(os.path.join(finalPath,'GLO_NoNorm/checkpoints'))
paths.append(os.path.join(finalPath,'GLO_NF_joint/Res128_GLO_NF_lambdae-3'))
paths.append(os.path.join(finalPath,'GLO_NF_ALTERNATE'))

In [33]:
fidFiles = []
fidFiles.append('FID_constant_interpolated_linear_sampling_latent.json')
fidFiles.append('FID_constant_interpolated_None_sampling_latent.json')
fidFiles.append('FID_sampled_interpolated_linear_sampling_latent.json')
fidFiles.append('FID_sampled_interpolated_None_sampling_latent.json')

In [36]:
modelFIDS = np.zeros((4,5))
i=0
j=0
for fid in fidFiles:
    j=0
    for modelPath in paths:
        fidVal = printJSON(os.path.join(modelPath, fid))
        modelFIDS[i,j] = fidVal
        print(modelPath + ' with FID: ' + fid + ' '  + str(fidVal))
        j+=1
    
    i+=1

{'FID': 29.466691970825195}
/home/matthias/ETH/Thesis/Euler/Final_Models/default/checkpoints with FID: FID_constant_interpolated_linear_sampling_latent.json 29.466691970825195
{'FID': 29.182600021362305}
/home/matthias/ETH/Thesis/Euler/Final_Models/GLO_Norm/checkpoints with FID: FID_constant_interpolated_linear_sampling_latent.json 29.182600021362305
/home/matthias/ETH/Thesis/Euler/Final_Models/GLO_NoNorm/checkpoints with FID: FID_constant_interpolated_linear_sampling_latent.json NaN
{'FID': 24.900615692138672}
/home/matthias/ETH/Thesis/Euler/Final_Models/GLO_NF_joint/Res128_GLO_NF_lambdae-3 with FID: FID_constant_interpolated_linear_sampling_latent.json 24.900615692138672
{'FID': 677.7301025390625}
/home/matthias/ETH/Thesis/Euler/Final_Models/GLO_NF_ALTERNATE with FID: FID_constant_interpolated_linear_sampling_latent.json 677.7301025390625
{'FID': 25.726268768310547}
/home/matthias/ETH/Thesis/Euler/Final_Models/default/checkpoints with FID: FID_constant_interpolated_None_sampling_late

In [39]:
modelFIDS.shape

(4, 5)

In [44]:
np.round(3.4521, 3)

3.452

In [55]:
def tableToLatex(arr):
    Models = np.array(['Baseline', 'GLO_Norm', 'GLO_NoNorm', 'Joint', 'Alternate'])
    cols, rows = arr.shape
    table = ''
    for i in range(rows):
        row = Models[i]
        for j in range(cols):
            row+=' & ' + str(np.round(arr[j,i],3))
        print(row + ' \\\\')
        

In [56]:
tableToLatex(modelFIDS)

Baseline & 29.467 & 25.726 & 18.817 & 17.205 \\
GLO_Norm & 29.183 & 29.183 & 20.506 & 20.135 \\
GLO_NoNorm & nan & nan & nan & nan \\
Joint & 24.901 & 24.901 & 17.375 & 17.43 \\
Alternate & 677.73 & 677.73 & 677.73 & 677.73 \\


In [38]:
modelFIDS

array([[ 29.46669197,  29.18260002,          nan,  24.90061569,
        677.73010254],
       [ 25.72626877,  29.18260002,          nan,  24.90061569,
        677.73010254],
       [ 18.81654167,  20.50551796,          nan,  17.37524033,
        677.73010254],
       [ 17.20462036,  20.13464355,          nan,  17.43047714,
        677.73010254]])

In [10]:
paths[0]

'/home/matthias/ETH/Thesis/Euler/Final_Models/default/checkpoints'

In [ ]:
shape = 'something'
if(not(shape ==  None)):
    print('True')
else:
    print(False)

In [ ]:
print(vals.opt.experiment_name)


In [ ]:
#out = vals.sample()
vals.inference_ffhq(opt_new, n=10000)
# psnr = np.array(vals.psnr).mean()
# ssim = np.array(vals.ssim).mean()
# lpips = np.array(vals.lpips).mean()
fid_std = vals.fid_std
print(fid_std)
# print('PSNR: ' + str(psnr), 'SSIM: '+ str(ssim), 'LPIPS: ' + str(lpips))

In [ ]:
validate(model, metric, )

In [ ]:
print(psnr, ssim, lpips, vals.fid_std())

In [ ]:
fid  = vals.metric_fid_std.fid.compute().item()

In [ ]:
print(fid)

In [ ]:
out = do_validation('norm', opt)

In [ ]:
idx = 0
for i, batch in enumerate(vals.dataloader):
    real = batch[DIK.IMAGE_IN]
    demo.visualizer_complete.tensor2image(real[0].squeeze(), return_format='pil')
    if(idx==i):
        break

demo.visualizer_complete.tensor2image(real[1].squeeze(), return_format='pil')

In [ ]:
demo.visualizer_complete.tensor2image(real[0].squeeze(), return_format='pil')

In [ ]:

t = torch.rand(1).to(vals.device)
print(t)

In [ ]:
t = torch.rand(1)
print(t)

In [ ]:
def slerp(a, b, t):
    a = a / a.norm(dim=-1, keepdim=True)
    b = b / b.norm(dim=-1, keepdim=True)
    d = (a * b).sum(dim=-1, keepdim=True)
    p = t * torch.acos(d)
    c = b - d * a
    c = c / c.norm(dim=-1, keepdim=True)
    d = a * torch.cos(p) + c * torch.sin(p)
    d = d / d.norm(dim=-1, keepdim=True)
    return d

In [ ]:
demo.visualizer_complete.tensor2image(slerp(real[0],real[1],0.01).squeeze(), return_format='pil')

In [ ]:
demo.visualizer_complete.tensor2image(i[idx].squeeze(), return_format='pil')

In [ ]:
demo.visualizer_complete.tensor2image(o[idx].squeeze(), return_format='pil')

In [ ]:
latent = demo.pipeline.Z.weight
print(latent[2376])
print(latent[idcs])

In [ ]:
idcs = torch.randint(0,vals.dataset.N-2, size=(7,))
print(idcs)

In [ ]:
vals.dataloader.dataset[0]
vals.dataloader.dataset.get_unsqueezed(0)

In [ ]:
clatent = latent[0:7]
clatent.shape

In [ ]:
def getBatches(idcs):
    

In [ ]:
print(batch[DIK.IMAGE_IN].shape)

In [ ]:
batch = vals.getBatch(mode='sampleVal')

batch[DIK.STYLE_LATENT]=clatent

batch = vals.model.forward(batch, 0)
img = batch[DIK.IMAGE_OUT]
demo.visualizer_complete.tensor2image(img[2].squeeze(), return_format='pil')